In [32]:
import numpy as np
import matplotlib.pyplot as mplot
import tensorflow as tf
import tensorflow_addons as tfa
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import os

In [3]:
from gan_load import file_load

In [6]:
path = 'data/'
dirs = os.listdir(path)
monet_path = path + dirs[0] + '/'
reg_path = path + dirs[1] + '/'

In [7]:
file_loader = file_load()

In [8]:
monet_img = file_loader.get_img(monet_path)

In [10]:
reg_img = file_loader.get_img(reg_path)

In [107]:
from tensorflow.keras.layers import Input, Conv2D, Activation,Add, LeakyReLU, Conv2DTranspose
from tensorflow.keras.models import Model


In [126]:
def R256(input_layer, res_position, f=256, k=(3,3), s=(1,1)):
    gaussian = tf.random_normal_initializer(0,0.02)
    
    x = Conv2D(filters=f, kernel_size=k, padding='same', strides=s, name='res'+str(res_position)+'a', kernel_initializer=gaussian)(input_layer)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=f, kernel_size=k, padding='same', strides=s, name='res'+str(res_position)+'b', kernel_initializer=gaussian)(x)
    x = Activation('relu')(x)
    
    x = Add(name = 'res'+str(res_position)+'final')([input_layer, x])
    
    return x
    
    
    

In [127]:
def pre_convolutional_block(input_layer):
    gaussian = tf.random_normal_initializer(0,0.02)
    
    x = Conv2D(filters=64, padding='same', kernel_size=(7,7), strides=(1,1), kernel_initializer=gaussian, name='conv1_pre')(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=128, padding='same', kernel_size=(3,3), strides=(2,2), kernel_initializer=gaussian, name='conv2_pre')(x)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Conv2D(filters=256, padding='same', kernel_size=(3,3), strides=(2,2), kernel_initializer=gaussian, name='conv3_pre')(x)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    return x

    

In [135]:
def post_convolutional_block(input_layer):
    gaussian = tf.random_normal_initializer(0,0.02)
    
    x = Conv2DTranspose(filters=128, kernel_size=(3,3), padding='same', strides=(2, 2), kernel_initializer=gaussian, name='conv1_post')(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Conv2DTranspose(filters=64, kernel_size=(3,3), padding='same', strides=(2, 2), kernel_initializer=gaussian, name='conv2_post')(x)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    x = Conv2DTranspose(filters=3, kernel_size=(7,7), padding='same', strides=(1, 1), kernel_initializer=gaussian, name='conv3_post')(x)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('relu')(x)
    
    return x

In [136]:
def make_generator(input_shape):
    
    input_layer = Input(input_shape)
    x = pre_convolutional_block(input_layer)
    x = R256(x, 1)
    x = R256(x,2)
    x = R256(x,3)
    x = R256(x, 4)
    x = R256(x,5)
    x = R256(x,6)
    x = R256(x, 7)
    x = R256(x,8)
    x = R256(x,9)
    x = post_convolutional_block(x)
    
    model = Model(inputs=input_layer, outputs=x)
    return model

In [137]:
generator_M = make_generator((256,256,3))

In [138]:
generator_R = make_generator((256,256,3))

In [139]:
generator_M.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv1_pre (Conv2D)              (None, 256, 256, 64) 9472        input_42[0][0]                   
__________________________________________________________________________________________________
instance_normalization_80 (Inst (None, 256, 256, 64) 128         conv1_pre[0][0]                  
__________________________________________________________________________________________________
activation_250 (Activation)     (None, 256, 256, 64) 0           instance_normalization_80[0][0]  
____________________________________________________________________________________________

In [134]:
def make_discriminator(input_shape):
    gaussian = tf.random_normal_initializer(0,0.02)
    
    input_layer = Input(input_shape)
    x = Conv2D(filters=64, kernel_size=(4,4), padding='same', strides=(2,2), name='disc_conv_1', kernel_initializer=gaussian)(input_layer)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters=128, kernel_size=(4,4), padding='same', strides=(2,2), name='disc_conv_2', kernel_initializer=gaussian)(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters=256, kernel_size=(4,4), padding='same', strides=(2,2), name='disc_conv_3', kernel_initializer=gaussian)(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters=512, kernel_size=(4,4), padding='same', strides=(2,2), name='disc_conv_4', kernel_initializer=gaussian)(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(filters=1, kernel_size=(4,4), padding='same', strides=(1,1), name='disc_conv_final', kernel_initializer=gaussian)(input_layer)
    x = tfa.layers.InstanceNormalization(axis=-1)(x)
    x = Activation('sigmoid')(x)
    
    
    